In [1]:
import pandas as pd

In [50]:
cs = pd.read_csv('/content/crime_scene_report.csv')
dl = pd.read_csv('/content/drivers_license.csv')
fec = pd.read_csv('/content/facebook_event_checkin.csv')
gfnci = pd.read_csv('/content/get_fit_now_check_in.csv')
gfnm = pd.read_csv('/content/get_fit_now_member.csv')
income = pd.read_csv('/content/income.csv')
interview = pd.read_csv('/content/interview.csv')
person = pd.read_csv('/content/person.csv')

In [6]:
our_murder = cs[(cs['date'] == 20180115) & (cs['type'] == 'murder') & (cs['city'] == 'SQL City')]

In [8]:
print(our_murder.loc[1227, 'description'])

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


In [9]:
first_w = person[(person['address_street_name'] == 'Northwestern Dr')].sort_values(by = 'address_number', ascending = False).head(1)

In [10]:
first_w

,id,name,license_id,address_number,address_street_name,ssn
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [11]:
second_w = person[(person['address_street_name'] == 'Franklin Ave') & (person['name'].str.contains('Annabel'))]

In [12]:
second_w

,id,name,license_id,address_number,address_street_name,ssn
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [13]:
witnesses = pd.merge(person, interview, left_on = 'id', right_on = 'person_id')

In [17]:
first_w_testimony = witnesses[(witnesses['name'] == 'Annabel Miller')].loc[321, 'transcript']
second_w_testimony = witnesses[(witnesses['name'] == 'Morty Schapiro')].loc[252, 'transcript']

In [18]:
first_w_testimony

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

In [19]:
second_w_testimony

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [20]:
gym_date = gfnci[gfnci['check_in_date'] == 20180109]

In [22]:
people_from_gym = pd.merge(gym_date, gfnm, left_on = 'membership_id', right_on = 'id')

In [23]:
suspects = people_from_gym[people_from_gym['membership_id'].str.startswith('48Z')]

In [24]:
suspects

,membership_id,check_in_date,check_in_time,check_out_time,id,person_id,name,membership_start_date,membership_status
7,48Z7A,20180109,1600,1730,48Z7A,28819,Joe Germuska,20160305,gold
8,48Z55,20180109,1530,1700,48Z55,67318,Jeremy Bowers,20160101,gold


In [26]:
cars = dl[dl['plate_number'].str.contains('H42W')]

In [35]:
sus_cars = pd.merge(cars, person, left_on = 'id', right_on = 'license_id')

In [38]:
sus_cars[(sus_cars['name'] == 'Jeremy Bowers') | (sus_cars['name'] == 'Joe Germuska')]

,id_x,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id_y,name,license_id,address_number,address_street_name,ssn
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [39]:
killer = 'Jeremy Bowers'

In [43]:
witnesses[witnesses['name'] == killer].loc[3152, 'transcript']

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

In [45]:
fec['date'] = fec['date'].astype(str)
new_df = fec[(fec['event_name'] == 'SQL Symphony Concert') & (fec['date'].str.contains('201712'))]
people = new_df.groupby('person_id')['date'].count()
people = people[people > 2]
pd.merge(people, person, left_on ='person_id', right_on = 'id')


,date,id,name,license_id,address_number,address_street_name,ssn
0,3,24556,Bryan Pardo,101191,703,Machine Ln,816663882
1,3,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [46]:
the_one_who_hired_the_killer = person[person['id'] == 99716]

In [47]:
the_one_who_hired_the_killer

,id,name,license_id,address_number,address_street_name,ssn
9985,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [49]:
#Убийца - Jeremy Bowers
#Заказчик - Miranda Priestly